<a href="https://colab.research.google.com/github/pedrotorres08/DistrReliability/blob/main/MFR_Allocation_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1- Packaging Imports

In [1]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from copy import deepcopy
import pandas as pd
import urllib
import json
import numpy as np

In [2]:
#Graph Visualization
!pip install -q pyvis

from pyvis.network import Network
from IPython.core.display import display, HTML

##2- Functions

In [3]:
def var_check(ti,tmn,tmx):
  if ti>tmx:
    return tmx
  if ti<tmn:
    return tmn
  else:
    return ti

def find_sub_path(n):
    for x in dic_sp:
      if n in x:
        n0 = x[n][0]
        n_path = x[n]
        n_path_rev = deepcopy(n_path)
        n_path_rev.reverse()
        edg_path = [(n_path_rev[k-1],n_path_rev[k]) for k in range(1,len(n_path_rev))]
        break
    return [n0, n_path_rev, edg_path]

def find_p_dev(G,edg_path):
    for dev in edg_path:
      tipo = G.edges[dev]['TIP_DEV']
      if tipo in devices['prot']:
        p_dev = dev
        break
    return p_dev

#Function to check if nh is transferable
def check_transfer(G,nf,nh):
    #Remove edges connected to nf and island the MG
    edges_to_remove = list(G.edges(nf))
    attrs_to_recover = {x:G.edges[x] for x in edges_to_remove}
    G.remove_edges_from(edges_to_remove)

    #Check if nh has path to substation
    check_has_path = list(nx.all_simple_edge_paths(G, nh, n0_list))
      
    #Recover original graph
    G.add_edges_from(edges_to_remove)
    nx.set_edge_attributes(G, attrs_to_recover)

    #For each n0 check if has path after isolate nf
    if check_has_path == []:
      #No
      transferable = False
    else:
      transferable = True   
    return transferable

#Function to check if nh is transferable to the microgrid
# eIS ->  edge (sectionalizing and prot. dev.) to island the MG
# nMFR -> equiv. node where the MFR is located
def check_transfer_mg(G,nf,nh):
    #If a MFR exists:
    if eIS != []:
      #Remove edges connected to nf and island the MG
      edges_to_remove = list(G.edges(nf)) + eIS
      attrs_to_recover = {x:G.edges[x] for x in edges_to_remove}
      G.remove_edges_from(edges_to_remove)

      #Check if nh has path to MFR
      check_has_path = nx.has_path(G, source=nMFR, target=nh)
      
      #Recover original graph
      G.add_edges_from(edges_to_remove)
      nx.set_edge_attributes(G, attrs_to_recover)
      
      #Check if has path between nh and MFR after islanding the MG
      if check_has_path:
        #If path exists, nh can operate islanded in MG
        transferable_mg = True
      else:
        #Otherwise, no.
        transferable_mg = False
    else:
     transferable_mg = False
    
    return transferable_mg

def SAIDI_contr(G,n_state,nf,nh,edg_path):
  if n_state == '1': #Not Affected
    return 0
  else:
    #Determines the prot device immediate upstream to nf
    for edge in edg_path:
      if G.edges[edge]['TIP_DEV'] in devices['prot']:
        up_prot_dev = G.edges[edge]['TIP_DEV']
        break

    #Verify if the prot device immediate upstream to nf has reclosing capacity
    imd_up_dev = up_prot_dev in devices['rel']
    
    #Check whether exists an recloser w/ coordination capacity in nf path
    dev_rel_coord = any(x in devices['coord'] for x in [G.edges[dev]['TIP_DEV'] for dev in edg_path])

    if n_state == '5': #Permanently Interrupted
      if imd_up_dev or dev_rel_coord:
        return r_fail[nf]*f_perm[nf]*n_cons[nh]
      else:
        return r_fail[nf]*n_cons[nh]
    
    if n_state == '2': #Recoverable
      if imd_up_dev or dev_rel_coord:
        return r_fail[nf]*f_perm[nf]*n_cons[nh]
      else:
        return r_fail[nf]*n_cons[nh]

    if n_state == '3': #Transferable
      if imd_up_dev or dev_rel_coord:
        return r_fail[nf]*f_perm[nf]*n_cons[nh]
      else:
        return r_fail[nf]*n_cons[nh]

    if n_state == '4': #Islanded in Microgrid
      #It is an assumption that the MG enters islanded mode fast enough so no
      #interruption is recorded
      return 0

def NFE(G,nf,nh):
  #Determine the substation node and its path to the faulty and healthy nodes
  [n0_nf,n_path_bf,edg_path_nf] = sub_vars[nf]
  [n0_nh,n_path_bv,edg_path_nh] = sub_vars[nh]
  
  #If node is the faulty node itself, it is permanently interrupted
  if nf == nh:
    n_state = '5' #Permanently Interrupted
    FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
    return n_state,FI
  else:
    #If the substation node of nh is different from the substation node of nf:
    if n0_nf != n0_nh:
      n_state = '1' #Not Affected
      FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
      return n_state,FI
    #Otherwise, check which protection device (p_dev) will trigger
    else:
      p_dev = find_p_dev(G,edg_path_nf)
      #Check if the triggered p_dev is in nh supply path
      if p_dev in edg_path_nh:
        #Check wether is possible to recover path
        if edg_path_nf[0] in edg_path_nh:
          #It is not possible to recover path
          #Check whether is possible to transfer nh to an alternative path
          if check_transfer(G,nf,nh):
            n_state = '3' #Transferable
            FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
            return n_state,FI
          else:
            #Check whether nh can be transfered to operate in islanded MG
            if check_transfer_mg(G,nf,nh):
              n_state = '4' #Islanded in Microgrid
              FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
              return n_state,FI
            else:
              n_state = '5' #Permanently Interrupted
              FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
              return n_state,FI
        else:
          #nh supply path can be recovered:
          n_state = '2' #Recoverable
          FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
          return n_state,FI
      #nh is not affected if trigered p_dev is not in nh path
      else:
        n_state = '1' #Not Affected
        FI = SAIDI_contr(G,n_state,nf,nh,edg_path_nf)
        return n_state,FI

def outage_dur(Mf,nf,nh):
  if Mf[nf][nh]['state'] == '1': #Not Affected
    DI = 0
    return DI
  
  if Mf[nf][nh]['state'] == '2': #Recoverable
    DI = (er_times[nf]['utL'] + er_times[nf]['utT']) * Mf[nf][nh]['fi']
    return DI
  
  if Mf[nf][nh]['state'] == '4': #Islanded in Microgrid
    DI = mg_isld_time * Mf[nf][nh]['fi']
    return DI
  
  #Check if has any node with recoverable state in row
  has_state_2 = any(Mf[nf][x]['state'] == '2' for x in Gnosub.nodes)

  #Check if has any node with transferable state in row
  has_state_3 = any(Mf[nf][x]['state'] == '3' for x in Gnosub.nodes)

  if Mf[nf][nh]['state'] == '3': #Transferable
    if (has_state_2 and has_state_3):
      DI = (er_times[nf]['utL'] + er_times[nf]['utT'] + er_times[nf]['utT']) * Mf[nf][nh]['fi']
      return DI
    else:
      DI = (er_times[nf]['utL'] + er_times[nf]['utT']) * Mf[nf][nh]['fi']
      return DI

  if Mf[nf][nh]['state'] == '5': #Permanently Interrupted
    if not(has_state_2 or has_state_3):
      DI = (er_times[nf]['utL'] + er_times[nf]['utR']) * Mf[nf][nh]['fi']
      return DI
    else:
      if (has_state_2 and has_state_3):
        DI = (er_times[nf]['utL'] + er_times[nf]['utT'] + er_times[nf]['utT'] + er_times[nf]['utR']) * Mf[nf][nh]['fi']
        return DI
      else:
        DI = (er_times[nf]['utL'] + er_times[nf]['utT'] + er_times[nf]['utR']) * Mf[nf][nh]['fi']
        return DI

def size_MFR(L,Dref,dt,pbase):
  #Calculate the MA of the load profiles @ interval u
  u = int(Dref/dt)
  len_L = len(L)

  L = pd.concat([L, L.iloc[0:u-1]], ignore_index=True)
  L['MA'] = L['equiv'].rolling(window=u).mean()

  L.iloc[0:u-1] = L.iloc[-u+1:]
  L = L.iloc[0:len_L]

  P_mfr = max(L['equiv'])*pbase
  E_mfr = max(L['MA'])*Dref*pbase 
  return P_mfr,E_mfr

## 3- MV Grid Graph Model (Example)

In [4]:
G = nx.Graph()

#Grid topology from Fig. 1(c)
#TIP_DEV -> Device type: B - Breaker, R - Recloser, F - Fuse Switch, 
#S - Sectionalizer, T - Tie Switch, FR - Fuse Swith w/ reclosing,
#D - Disconnect Switch
#P_N_OPE -> Nominal Operation Position: NC - Normally Closed, 
#NO - Normally Open
G.add_edges_from([("n0a", "n1", {'TIP_DEV': 'B','P_N_OPE': 'NC'}), 
                  ("n1", "n2", {'TIP_DEV': 'R','P_N_OPE': 'NC'}),
                  ("n2", "n3", {'TIP_DEV': 'F','P_N_OPE': 'NC'}),
                  ("n2", "n4", {'TIP_DEV': 'S','P_N_OPE': 'NC'}),
                  ("n5", "n6", {'TIP_DEV': 'F','P_N_OPE': 'NC'}),
                  ("n7", "n8", {'TIP_DEV': 'FR','P_N_OPE': 'NC'}),
                  ("n2", "n8", {'TIP_DEV': 'T','P_N_OPE': 'NO'}),
                  ("n1", "n7", {'TIP_DEV': 'T','P_N_OPE': 'NO'}),
                  ("n7", "n5", {'TIP_DEV': 'D','P_N_OPE': 'NC'}),
                  ("n6", "n9", {'TIP_DEV': 'D','P_N_OPE': 'NC'}),
                  ("n0b", "n5", {'TIP_DEV': 'B','P_N_OPE': 'NC'}),
                  ])

#Failure rate of each equiv. node [failures/yr]
r_fail = {'n1': 8.95, 'n2': 6.27, 'n3': 8.81, 'n4': 9.57, 'n5': 4.65,
          'n6': 8.73, 'n7': 3.79, 'n8': 7.87, 'n9': 3.79,} 

#Permanent fault factor of each equiv. node [permanent faults/ total faults]
f_perm = {'n1': 0.2, 'n2': 0.2, 'n3': 0.2, 'n4': 0.2, 'n5': 0.2,
          'n6': 0.2, 'n7': 0.2, 'n8': 0.2, 'n9': 0.2,}

#Number of consumers connected at each equiv. node
n_cons = {'n1': 62, 'n2': 9, 'n3': 27, 'n4': 48, 'n5': 113,
          'n6': 8, 'n7': 67, 'n8': 12, 'n9': 60,}

#Emergency Response Times
#utL - average fault localization time [h]
#utR - average repair time [h]
#utT - average circuit section transfer time [h]
er_times = {'n1':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n2':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n3':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n4':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n5':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n6':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n7':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n8':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},
            'n9':{'utL': 0.7, 'utR': 0.4, 'utT': 0.18},}

#Standard deviation for emergency response times
[std_tL,std_tR,std_tT] = [0.3,0.2,0.08]

#List of substation nodes
n0_list = ["n0a","n0b"]

#Protection Devices and their behavior
devices = {'prot': ['B','R','S','FR','F'], #Protection devices
           'rel': ['B','R','S','FR'], #Prot. dev. w/ reclosing capacity
           'coord': ['R']} #Prot. dev. w/ coordinated reclosing capacity

#Historical Indexes Data
SAIFImv_h = 3.0
SAIDImv_h = 3.5

## 4- NFEM, Adjustment from Historical Data and Reference Scenario Model

In [5]:
#No MFR at this momment             
nMFR = '' #node where the MFR is located
eIS = [] #edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

#Create graph Gnom which is G excluded NO edges
Gnom = deepcopy(G)
Gnom.remove_edges_from([k for k,v in nx.get_edge_attributes(G, "P_N_OPE").items() if v == 'NO'])

#Calculate the shortest paths from each equiv. node to the substation nodes
dic_sp = [nx.single_source_shortest_path(Gnom, source) for source in n0_list]

#Create graph Gnosub which is G excluded substation nodes
Gnosub = deepcopy(G)
Gnosub.remove_nodes_from(n0_list)

#Initialize NFEM with all elements as permanently interrupted
Mf = {j:{i: {'state':'5', 'fi':0, 'di':0} for i in Gnosub.nodes} for j in Gnosub.nodes}

#Find the substation nodes that are connected to each equiv. node and their
#respective path
sub_vars = {n: find_sub_path(n) for n in G.nodes}

#Populate NFEM and the expected failure rate associated to each element
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    [state,fi] = NFE(G,nf,nh)
    Mf[nf][nh]['state'] = state
    Mf[nf][nh]['fi'] = fi

#Failure rate calibration from historical data
SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("SAIFImv_c before adjustment = {:.2f}".format(SAIFImv_c))

f_adj = SAIFImv_h/SAIFImv_c
print("Failure rate adjust factor = {:.2f}".format(f_adj))

for n in Gnosub.nodes:
  r_fail[n] = round(r_fail[n]*f_adj,2)
print("Adjusted failure rates = {}".format(list(r_fail.values())))

#Vulnerability and Contribution w.r.t. SAIFI
v = [round(sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nh in Gnosub.nodes]
p = [round(sum([Mf[nf][nh]['fi'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIFI = {}".format(v))
print("Relative Nodal Contribution w.r.t. SAIFI = {}".format(p))

#Calculate the expected failure rate again using calibrated failure rate param.
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    E = Mf[nf][nh]['state']
    edg_path_nf = sub_vars[nf][2]
    Mf[nf][nh]['fi'] = SAIDI_contr(G,E,nf,nh,edg_path_nf)

SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Calculate the initial expected outage time (iteration 0)
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)

SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Vulnerability and Contribution w.r.t. SAIDI
w = [round(sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nh in Gnosub.nodes]
d = [round(sum([Mf[nf][nh]['di'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIDI = {}".format(w))
print("Relative Nodal Contribution w.r.t. SAIDI = {}".format(d))

#Calculate initial error in SAIDI calculation
eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)
print("\neSAIDI before adjust = {:.2f} %".format(eSAIDI*100))

#Emergency response time calibration from historical data
k_A = 0.1
eSAIDI_max = 0.01
it_mx = 100 #Maximum number of iterations

[utL_min,utL_max] = [0, 100]
[utR_min,utR_max] = [0, 100]
[utT_min,utT_max] = [0, 100]

print("\nIterative process:")
iter = 0
while abs(eSAIDI) > eSAIDI_max:
  print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

  #Update emergency response times
  for x in Gnosub.nodes:
    er_times[x]['utL'] = var_check(er_times[x]['utL'] - eSAIDI*std_tL*k_A,utL_min,utL_max)
    er_times[x]['utR'] = var_check(er_times[x]['utR'] - eSAIDI*std_tR*k_A,utR_min,utR_max)
    er_times[x]['utT'] = var_check(er_times[x]['utT'] - eSAIDI*std_tT*k_A,utT_min,utT_max)

  #Update expected outage time
  for nf in Gnosub.nodes:
    for nh in Gnosub.nodes:
      Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)
  
  #Update SAIDImv_c and eSAIDI
  SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
  eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)

  #Next iteration
  if iter < it_mx:
    iter = iter + 1
  else:
    break

print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

print("\nAdjusted SAIDImv_c = {:.2f}".format(SAIDImv_c))
print("Adjusted SAIFImv_c = {:.2f}".format(SAIFImv_c))

SAIFImv_c before adjustment = 4.47
Failure rate adjust factor = 0.67
Adjusted failure rates = [6.01, 4.21, 5.91, 6.42, 3.12, 5.86, 2.54, 5.28, 2.54]
Relative Nodal Vulnerability w.r.t. SAIFI = [0.06, 0.02, 0.07, 0.13, 0.11, 0.06, 0.06, 0.02, 0.47]
Relative Nodal Contribution w.r.t. SAIFI = [0.14, 0.06, 0.03, 0.05, 0.13, 0.33, 0.11, 0.01, 0.14]
Relative Nodal Vulnerability w.r.t. SAIDI = [0.07, 0.01, 0.07, 0.12, 0.1, 0.06, 0.06, 0.02, 0.48]
Relative Nodal Contribution w.r.t. SAIDI = [0.14, 0.06, 0.03, 0.05, 0.14, 0.32, 0.1, 0.01, 0.16]

eSAIDI before adjust = -4.63 %

Iterative process:
It. no. 0, utL = 0.7, utT = 0.18, utR = 0.4, SAIDImv_c = 3.34, eSAIDI = -0.0463
It. no. 1, utL = 0.7014, utT = 0.1804, utR = 0.4009, SAIDImv_c = 3.34, eSAIDI = -0.0443
It. no. 2, utL = 0.7027, utT = 0.1807, utR = 0.4018, SAIDImv_c = 3.35, eSAIDI = -0.0424
It. no. 3, utL = 0.704, utT = 0.1811, utR = 0.4027, SAIDImv_c = 3.36, eSAIDI = -0.0406
It. no. 4, utL = 0.7052, utT = 0.1814, utR = 0.4035, SAIDImv_c =

## 5- Reading Load data

In [6]:
#Read load data from github repo
url = "https://raw.githubusercontent.com/pedrotorres08/DistrReliability/main/load_data.csv"
load_df = pd.read_csv(url)

for n in Gnosub.nodes:
  #Assuming that all consumers have the same load profile
  load_df[n] = load_df['demand after DR program [p.u./cons]']*n_cons[n] 

pbase = 10 #base power in kW for p.u. conversion
Dref = 4 #mean service restoration time 
dt = 0.25 #time between samples in load curve, in hours (15 min => 0.25)

##6- MFR and IS Allocation<br>


###6.1 - Alternative I

In [7]:
#Place MFR and IS             
nMFR = 'n9' #node where the MFR is located
eIS = [('n6','n9')]#edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

Mf_mod = deepcopy(Mf)

#Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:          
    if (nf != nh):
      #If nh is permanently interrupted in the reference case
      if (Mf_mod[nf][nh]['state'] == '5'): 
        #If it is possible to transfer nh to operate in islanded MG
        if check_transfer_mg(G,nf,nh):
          Mf_mod[nf][nh]['state'] = '4'
          edg_path_nf = sub_vars[nf][2]
          Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

#Update expected outage time
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("nMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

#Calculate Power and Energy Requirements
#First, determine which equivalent nodes are part of the microgrid
Gisld = deepcopy(Gnom)
Gisld.remove_edge(*eIS[0]) 
mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
print("Nodes in MG: {}".format(mg_nodes))

#Then, calculate the equivalent load curve
load_df['equiv']= load_df[mg_nodes].sum(axis=1)

#Determine the Moving Average and Calculate Pmfr and Emfr
[P_mfr,E_mfr] = size_MFR(load_df,Dref,dt,pbase)
print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

#Calculate rSAIDI and rSAIFI
rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

nMFR: n9, eIS: [('n6', 'n9')]
SAIFImv_n: 2.04, SAIDImv_n = 2.35 h
Nodes in MG: ['n9']
Pmfr = 9.60 kW and Emfr = 34.50 kWh
rSAIDI = 0.93 %/kWh and rSAIFI = 0.93 %/kWh


###6.2 - Alternative II

In [8]:
#Place MFR and IS             
nMFR = 'n6' #node where the MFR is located
eIS = [('n5','n6')] #edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

Mf_mod = deepcopy(Mf)

#Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:          
    if (nf != nh):
      #If nh is permanently interrupted in the reference case
      if (Mf_mod[nf][nh]['state'] == '5'): 
        #If it is possible to transfer nh to operate in islanded MG
        if check_transfer_mg(G,nf,nh):
          Mf_mod[nf][nh]['state'] = '4'
          edg_path_nf = sub_vars[nf][2]
          Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

#Update expected outage time
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("nMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

#Calculate Power and Energy Requirements
#First, determine which equivalent nodes are part of the microgrid
Gisld = deepcopy(Gnom)
Gisld.remove_edge(*eIS[0]) 
mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
print("Nodes in MG: {}".format(mg_nodes))

#Then, calculate the equivalent load curve
load_df['equiv']= load_df[mg_nodes].sum(axis=1)

#Determine the Moving Average and Calculate Pmfr and Emfr
[P_mfr,E_mfr] = size_MFR(load_df,Dref,dt,pbase)
print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

#Calculate rSAIDI and rSAIFI
rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

nMFR: n6, eIS: [('n5', 'n6')]
SAIFImv_n: 2.90, SAIDImv_n = 3.33 h
Nodes in MG: ['n6', 'n9']
Pmfr = 10.88 kW and Emfr = 39.10 kWh
rSAIDI = 0.10 %/kWh and rSAIFI = 0.09 %/kWh


###6.3 - Alternative III

In [9]:
#Place MFR and IS             
nMFR = 'n9' #node where the MFR is located
eIS = [('n5','n6')] #edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

Mf_mod = deepcopy(Mf)

#Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:          
    if (nf != nh):
      #If nh is permanently interrupted in the reference case
      if (Mf_mod[nf][nh]['state'] == '5'): 
        #If it is possible to transfer nh to operate in islanded MG
        if check_transfer_mg(G,nf,nh):
          Mf_mod[nf][nh]['state'] = '4'
          edg_path_nf = sub_vars[nf][2]
          Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

#Update expected outage time
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("nMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

#Calculate Power and Energy Requirements
#First, determine which equivalent nodes are part of the microgrid
Gisld = deepcopy(Gnom)
Gisld.remove_edge(*eIS[0]) 
mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
print("Nodes in MG: {}".format(mg_nodes))

#Then, calculate the equivalent load curve
load_df['equiv']= load_df[mg_nodes].sum(axis=1)

#Determine the Moving Average and Calculate Pmfr and Emfr
[P_mfr,E_mfr] = size_MFR(load_df,Dref,dt,pbase)
print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

#Calculate rSAIDI and rSAIFI
rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

nMFR: n9, eIS: [('n5', 'n6')]
SAIFImv_n: 2.03, SAIDImv_n = 2.34 h
Nodes in MG: ['n9', 'n6']
Pmfr = 10.88 kW and Emfr = 39.10 kWh
rSAIDI = 0.83 %/kWh and rSAIFI = 0.83 %/kWh


###6.4 - Alternative IV

In [10]:
#Place MFR and IS             
nMFR = 'n5' #node where the MFR is located
eIS = [('n0b','n5')]#edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

Mf_mod = deepcopy(Mf)

#Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:          
    if (nf != nh):
      #If nh is permanently interrupted in the reference case
      if (Mf_mod[nf][nh]['state'] == '5'): 
        #If it is possible to transfer nh to operate in islanded MG
        if check_transfer_mg(G,nf,nh):
          Mf_mod[nf][nh]['state'] = '4'
          edg_path_nf = sub_vars[nf][2]
          Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

#Update expected outage time
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("nMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

#Calculate Power and Energy Requirements
#First, determine which equivalent nodes are part of the microgrid
Gisld = deepcopy(Gnom)
Gisld.remove_edge(*eIS[0]) 
mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
print("Nodes in MG: {}".format(mg_nodes))

#Then, calculate the equivalent load curve
load_df['equiv']= load_df[mg_nodes].sum(axis=1)

#Determine the Moving Average and Calculate Pmfr and Emfr
[P_mfr,E_mfr] = size_MFR(load_df,Dref,dt,pbase)
print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

#Calculate rSAIDI and rSAIFI
rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

nMFR: n5, eIS: [('n0b', 'n5')]
SAIFImv_n: 3.00, SAIDImv_n = 3.47 h
Nodes in MG: ['n5', 'n6', 'n7', 'n9', 'n8']
Pmfr = 41.60 kW and Emfr = 149.50 kWh
rSAIDI = 0.00 %/kWh and rSAIFI = 0.00 %/kWh


##7- Case Studies

In [11]:
#First, get Hardware info
!cat /proc/cpuinfo # CPU info
!cat /proc/meminfo # Memory info

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

###7.1- Sparce and Remote Off-Grid System (SSB Substation)

####7.1.1- Reading Graph from Database

In [12]:
#Read substation equivalent graph data from github repo
url = 'https://raw.githubusercontent.com/LSF-IEE/DistRel/main/SSB.json'
with urllib.request.urlopen(url) as page:
    G = nx.node_link_graph(json.load(page))

#Number of consumers connected at each equiv. node (acquired from utility database)
n_cons = {x:G.nodes[x]['nuc_bt'] + G.nodes[x]['nuc_mt'] for x in G.nodes}

#Failure rate of each equiv. node [failures/yr] (acquired from utility database)
r_fail = {x:G.nodes[x]['tf'] for x in G.nodes}

#Permanent fault factor of each equiv. node [permanent faults/ total faults] (acquired from utility database)
f_perm = {x:G.graph['fator_fp'] for x in G.nodes}

#Emergency Response Times
#utL - average fault localization time [h]
#utR - average repair time [h]
#utT - average circuit section transfer time [h]
er_times = {x:{'utL': (G.graph['TMP']+G.graph['TMD']), 'utR': G.graph['TME'], 'utT': G.graph['TME']} for x in G.nodes}

#List of substation nodes (acquired from utility database)
n0_list = [x[0] for x in nx.get_node_attributes(G, "pac").items() if x[1] == True] 

#Definition of protection devices
devices = {'prot': ['22','29','32','33','34','35','36'], #Protection devices
                'rel': ['29','32','33','34','35','36'], #Prot. dev. w/ reclosing capacity
                'coord': ['32']} #Prot. dev. w/ coordinated reclosing capacity

#Standard deviation for emergency response times (acquired from utility database)
[std_tL,std_tR,std_tT] = [G.graph['TMPstd'],G.graph['TMDstd'],G.graph['TMDstd']]

#Minumum and maximum values for emergency response times (acquired from utility database)
[utL_min,utL_max] = [0.1*(G.graph['TMPmin'] + G.graph['TMDmin']), 1.3*(G.graph['TMPmax'] + G.graph['TMDmax'])]
[utR_min,utR_max] = [0.7*G.graph['TMEmin'], 1.3*G.graph['TMEmax']]
[utT_min,utT_max] = [0.7*G.graph['TMEmin'], 1.3*G.graph['TMEmax']]

#Historical Indexes Data (acquired from utility database)
SAIFImv_h = G.graph['FECp_h']
SAIDImv_h = G.graph['DECp_h']

Dref = G.graph['TMP'] + G.graph['TMD'] + G.graph['TME'] #mean service restoration time (acquired from utility database)
pbase = 1 #base power in kW for p.u. conversion (from load data acquired from utility database)
dt = 0.25 #time between samples in load curve, in hours (15 min => 0.25)
number_load_points = 96

####7.1.2 - Calculate Reference Scenario

In [13]:
#No MFR at this momment             
nMFR = '' #node where the MFR is located
eIS = [] #edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

#Create graph Gnom which is G excluded NO edges
Gnom = deepcopy(G)
Gnom.remove_edges_from([k for k,v in nx.get_edge_attributes(G, "P_N_OPE").items() if v == 'NO'])

#Remove isolated nodes (necessary fue to error in utility database)
G.remove_nodes_from([node for (node, val) in Gnom.degree() if val == 0])

#Calculate the shortest paths from each equiv. node to the substation nodes
dic_sp = [nx.single_source_shortest_path(Gnom, source) for source in n0_list]

#Create graph Gnosub which is G excluded substation nodes
Gnosub = deepcopy(G)
Gnosub.remove_nodes_from(n0_list)

#Initialize NFEM with all elements as permanently interrupted
Mf = {j:{i: {'state':'5', 'fi':0, 'di':0} for i in Gnosub.nodes} for j in Gnosub.nodes}

#Find the substation nodes that are connected to each equiv. node and their
#respective path
sub_vars = {n: find_sub_path(n) for n in G.nodes}

#Populate NFEM and the expected failure rate associated to each element
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    [state,fi] = NFE(G,nf,nh)
    Mf[nf][nh]['state'] = state
    Mf[nf][nh]['fi'] = fi

#Failure rate calibration from historical data
SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("SAIFImv_c before adjustment = {:.2f}".format(SAIFImv_c))

f_adj = SAIFImv_h/SAIFImv_c
print("Failure rate adjust factor = {:.2f}".format(f_adj))

for n in Gnosub.nodes:
  r_fail[n] = round(r_fail[n]*f_adj,2)
print("Adjusted failure rates = {}".format(list(r_fail.values())))

#Vulnerability and Contribution with respect to SAIFI
v = [round(sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nh in Gnosub.nodes]
p = [round(sum([Mf[nf][nh]['fi'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIFI = {}".format(v))
print("Relative Nodal Contribution w.r.t. SAIFI = {}".format(p))

#Calculate the expected failure rate again using calibrated failure rate param.
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    E = Mf[nf][nh]['state']
    edg_path_nf = sub_vars[nf][2]
    Mf[nf][nh]['fi'] = SAIDI_contr(G,E,nf,nh,edg_path_nf)

SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Calculate the initial expected outage time (iteration 0)
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)

SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Vulnerability and Contribution with respect to SAIDI
w = [round(sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nh in Gnosub.nodes]
d = [round(sum([Mf[nf][nh]['di'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIDI = {}".format(w))
print("Relative Nodal Contribution w.r.t. SAIDI = {}".format(d))

#Calculate initial error in SAIDI calculation
eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)
print("\neSAIDI before adjust = {:.2f} %".format(eSAIDI*100))

#Emergency response time calibration from historical data
k_A = 0.1
eSAIDI_max = 0.01
it_mx = 100 #Maximum number of iterations

print("\nIterative process:")
iter = 0
while abs(eSAIDI) > eSAIDI_max:
  print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

  #Update emergency response times
  for x in Gnosub.nodes:
    er_times[x]['utL'] = var_check(er_times[x]['utL'] - eSAIDI*std_tL*k_A,utL_min,utL_max)
    er_times[x]['utR'] = var_check(er_times[x]['utR'] - eSAIDI*std_tR*k_A,utR_min,utR_max)
    er_times[x]['utT'] = var_check(er_times[x]['utT'] - eSAIDI*std_tT*k_A,utT_min,utT_max)

  #Update expected outage time
  for nf in Gnosub.nodes:
    for nh in Gnosub.nodes:
      Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)
  
  #Update SAIDImv_c and eSAIDI
  SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
  eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)

  #Next iteration
  if iter < it_mx:
    iter = iter + 1
  else:
    break

print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

print("\nAdjusted SAIDImv_c = {:.2f}".format(SAIDImv_c))
print("Adjusted SAIFImv_c = {:.2f}".format(SAIFImv_c))

SAIFImv_c before adjustment = 4.25
Failure rate adjust factor = 1.87
Adjusted failure rates = [32.58, 40.74, 7.54, 1.61, 16.1, 32.49, 14.73, 11.07, 1.01, 19.9, 8.79, 0.16, 13.55, 2.71, 28.33, 5.72, 0.42, 4.59, 0.7, 0.25, 0.36, 1.9, 1.15, 0.03, 0.41, 0.38, 0.03, 0.39, 1.25, 2.57, 3.36, 0.21, 0.02, 0.15, 0.33, 2.02, 0.02, 0.08, 1.13, 0.96, 0.03, 0.03, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

####7.1.3- Calculate nodal contribution, vulnerability and generate communities

In [14]:
#Calculate the set of most critical nodes (R)
Cblkp = {x:{'SAIDImv':{'ab':0, '%': 0},'SAIFImv':{'ab':0, '%': 0}} for x in Gnosub.nodes}

for nf in Gnosub.nodes:
    Cblkp[nf]['SAIFImv']['ab'] = sum([Mf[nf][nh]['fi'] for nh in Gnosub.nodes])/sum(n_cons.values())
    Cblkp[nf]['SAIFImv']['%'] = Cblkp[nf]['SAIFImv']['ab']/SAIFImv_c*100
    Cblkp[nf]['SAIDImv']['ab'] = sum([Mf[nf][nh]['di'] for nh in Gnosub.nodes])/sum(n_cons.values())
    Cblkp[nf]['SAIDImv']['%'] = Cblkp[nf]['SAIDImv']['ab']/SAIDImv_c*100

dfRES = pd.DataFrame.from_dict({(i,j): Cblkp[i][j] for i in Cblkp.keys() for j in Cblkp[i].keys()}, orient='index').sort_values(by=['%'],ascending = False)
dfSAIDImv = dfRES[dfRES.index.get_level_values(1) == 'SAIDImv'].droplevel(1).reset_index()
dfSAIDImv['cum'] = dfSAIDImv['%'].cumsum()
idx_50 = dfSAIDImv[dfSAIDImv.cum >= 50].index[0]
n_crit_list = list(dfSAIDImv.loc[0:idx_50]['index'])
print("Set of most critical nodes (R): {}".format(n_crit_list))

#Calculate the set of most vulnerable nodes (U)
Vblkp = {x:{'SAIDImv':{'ab':0, '%': 0},'SAIFImv':{'ab':0, '%': 0}} for x in Gnosub.nodes}
for nh in Gnosub.nodes:
    Vblkp[nh]['SAIFImv']['ab'] = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes])/sum(n_cons.values())
    Vblkp[nh]['SAIFImv']['%'] = Vblkp[nh]['SAIFImv']['ab']/SAIFImv_c*100
    Vblkp[nh]['SAIDImv']['ab'] = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes])/sum(n_cons.values())
    Vblkp[nh]['SAIDImv']['%'] = Vblkp[nh]['SAIDImv']['ab']/SAIDImv_c*100

dfRES_vun = pd.DataFrame.from_dict({(i,j): Vblkp[i][j] for i in Vblkp.keys() for j in Vblkp[i].keys()}, orient='index').sort_values(by=['%'],ascending = False)
dfSAIDImv_vun = dfRES_vun[dfRES_vun.index.get_level_values(1) == 'SAIDImv'].droplevel(1).reset_index()
dfSAIDImv_vun['cum'] = dfSAIDImv_vun['%'].cumsum()
idx_50_vun = dfSAIDImv_vun[dfSAIDImv_vun.cum >= 50].index[0]
n_vun_list = list(dfSAIDImv_vun.loc[0:idx_50_vun]['index'])
print("Set of most vulnerable nodes (U): {}".format(n_vun_list))

#Divide the graph into communities using the CNM algorithm
communities = greedy_modularity_communities(G)

Set of most critical nodes (R): ['25447-7853701']
Set of most vulnerable nodes (U): ['25447-7854312CF', '25447-7853893CF', '25447-7853786CF', '12659-7854841', '25447-7854175CF', '25447-7854719CF', '25447-7854806CF', '25447-7854602CF', '25447-11418524CF', '25447-7853840CF', '25447-7853377CF', '25447-7854768CF', '25447-7854351CF', '25447-7853739CF', '12659-7854220CF', '25447-7853612CF', '25447-7854630', '25447-7853352CF', '25447-7854290CF', '25447-7853903CF', '25447-7854308CF', '25447-7853430CF', '25447-7853388CF', '25447-7854822CF', '25447-7853779CF', '25447-7853329CF', '25447-7854154CF', '25447-7854563CF', '25447-7853829CF', '25447-7853500']


####7.1.4- Graph Visualization

In [15]:
communities_dict = {}
shape_dict = {}

for i in range(len(communities)):
  for node in list(communities[i]):
    communities_dict[node] = i

for node in G.nodes:
  if G.nodes[node]['pac']:
    shape_dict[node] = 'star'
  else:
    if node in n_crit_list:
      shape_dict[node] = 'triangle'
    else:
      if node in n_vun_list:
        shape_dict[node] = 'square'
      else:
        shape_dict[node] = 'dot'

nx.set_node_attributes(G, communities_dict, name='group')
nx.set_node_attributes(G, shape_dict, name='shape')

nt = Network(notebook=True, cdn_resources='in_line', bgcolor='white',font_color="red")
nt.from_nx(G)
nt.show('{}-ref.html'.format('SSB'))
display(HTML('{}-ref.html'.format('SSB')))

SSB-ref.html


####7.1.5- Generate Scenarios for MFR allocation

In [16]:
nMFR_list = []
eIS_list = []

#For each critical node
for n_crit in n_crit_list:
  #Nodes in the n_crit community
  nodes_com_n_crit = set([x for x in communities if n_crit in x][0]) 
   
  for node in nodes_com_n_crit:
    #For each node in n_crit community, list its neighbors
    neigh_list = list(G.neighbors(node))
    for neigh in neigh_list:
      #If the neighbor is from another community, and is not a substation node, then the node is a possible nMFR, and node-neigh a possible eIS
      if not(neigh in nodes_com_n_crit):
        if not(neigh in sub_vars[n_crit][1]):
          nMFR_list.append(neigh)
          eIS_list.append((node,neigh))

  #Check if there is a vulnerable node in the critical node community
  for node in nodes_com_n_crit:
    if node in n_vun_list:
      #If existis, check if the supply path passes through the critical node
      if n_crit in sub_vars[node][1]:
        #If passes, determine the edge that leaves the critical node towards the vulnerable node
        nMFR_list.append(sub_vars[node][1][sub_vars[node][1].index(n_crit)-1])
        eIS_list.append((n_crit,sub_vars[node][1][sub_vars[node][1].index(n_crit)-1]))
      else:
        #If it doesn't passes
        for nd in sub_vars[node][1][1:]:
          neigh_list = list(G.neighbors(nd))
          for neigh in neigh_list:
            if not(neigh in nodes_com_n_crit):
              nMFR_list.append(neigh)
              eIS_list.append((nd,neigh))

#Remove duplicates
scen_df = pd.DataFrame.from_dict(dict(zip(nMFR_list, eIS_list)), orient='index').reset_index().drop_duplicates()
nMFR_list = list(scen_df['index'])
eIS_list = list((list(scen_df[0])[k],list(scen_df[1])[k]) for k in range(len(scen_df)))

####7.1.6- Run Reliability Assessment for each possible scenario

In [17]:
%%time

SAIFImv_n_list = []
SAIDImv_n_list = []
P_mfr_list = []
E_mfr_list = []
rSAIDI_list = []
rSAIFI_list = []
for edge,nMFR in zip(eIS_list,nMFR_list):  
  #First, check if the MFR/IS is a valid combination
  eIS  = [edge]
  if edge in sub_vars[nMFR][2] or edge[::-1]  in sub_vars[nMFR][2]:
    
    mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode
    Mf_mod = deepcopy(Mf)

    #Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
    for nf in Gnosub.nodes:
      for nh in Gnosub.nodes:          
        if (nf != nh):
          #If nh is permanently interrupted in the reference case
          if (Mf_mod[nf][nh]['state'] == '5'): 
            #If it is possible to transfer nh to operate in islanded MG
            if check_transfer_mg(G,nf,nh):
              Mf_mod[nf][nh]['state'] = '4'
              edg_path_nf = sub_vars[nf][2]
              Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

    #Update expected outage time
    for nf in Gnosub.nodes:
      for nh in Gnosub.nodes:
        Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

    SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
    SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
    SAIFImv_n_list.append(SAIFImv_n)
    SAIDImv_n_list.append(SAIDImv_n)
    print("\nnMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

    #Calculate Power and Energy Requirements
    #First, determine which equivalent nodes are part of the microgrid
    Gisld = deepcopy(Gnom)
    Gisld.remove_edge(*eIS[0]) 
    mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
    print("Nodes in MG: {}".format(mg_nodes))

    #Then, calculate the equivalent load curve
    eq_ld_curve = np.zeros(number_load_points)
    for node in mg_nodes:
      eq_ld_curve = eq_ld_curve + np.array(G.nodes[node]['eq_load'])
    
    eq_ld_curve = pd.DataFrame({'equiv':eq_ld_curve})

    #Determine the Moving Average and Calculate Pmfr and Emfr
    [P_mfr,E_mfr] = size_MFR(eq_ld_curve,Dref,dt,pbase)
    P_mfr_list.append(P_mfr)
    E_mfr_list.append(E_mfr)
    print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

    #Calculate rSAIDI and rSAIFI
    rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
    rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
    rSAIDI_list.append(rSAIDI)
    rSAIFI_list.append(rSAIFI)
    print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

  else:
    SAIFImv_n_list.append('')
    SAIDImv_n_list.append('')
    P_mfr_list.append('')
    E_mfr_list.append('')
    rSAIDI_list.append('')
    rSAIFI_list.append('')
    print("IS not in MFR's path!")

dfMG = pd.DataFrame({'nMFR': nMFR_list,'eIS': eIS_list,'SAIFImv_n': SAIFImv_n_list,'SAIDImv_n': SAIDImv_n_list, 'P_mfr': P_mfr_list, 'E_mfr': E_mfr_list, 'rSAIDI': rSAIDI_list, 'rSAIFI': rSAIFI_list})
dfMG['SAIDImv_n'] = pd.to_numeric(dfMG.SAIDImv_n, downcast='float')
dfMG['SAIFImv_n'] = pd.to_numeric(dfMG.SAIFImv_n, downcast='float')
dfMG = dfMG.dropna()

print('\nExecution Time:')


nMFR: 25447-7854587, eIS: [('25447-7853701', '25447-7854587')]
SAIFImv_n: 7.72, SAIDImv_n = 20.44 h
Nodes in MG: ['25447-7854587', '25447-7853779CF', '25447-7854120CF', '25447-7853741CF', '25447-7853840CF']
Pmfr = 8.91 kW and Emfr = 32.88 kWh
rSAIDI = 0.08 %/kWh and rSAIFI = 0.08 %/kWh

nMFR: 25447-7854100, eIS: [('25447-7853701', '25447-7854100')]
SAIFImv_n: 7.14, SAIDImv_n = 18.90 h
Nodes in MG: ['25447-7854100', '25447-11418564', '25447-11418395', '25447-7854736CF', '25447-7854734CF', '25447-7853329CF', '25447-7853554CF', '25447-7854296CF', '25447-7853922CF', '25447-7854228CF', '25447-7854100CF', '25447-11418535', '25447-11418597CF', '25447-11418531CF', '25447-11418606CF', '25447-11418624CF', '25447-11418564CF', '25447-11418566CF', '25447-11418571CF', '25447-11418607CF', '25447-11418623CF', '25447-11418507CF', '25447-11418513CF', '25447-11418581CF', '25447-11418548CF', '25447-11418576CF', '25447-11418550CF', '25447-11418614CF', '25447-11418611CF', '25447-11418618CF', '25447-1141856

In [18]:
dfMG.sort_values(by=['SAIDImv_n'])

,nMFR,eIS,SAIFImv_n,SAIDImv_n,P_mfr,E_mfr,rSAIDI,rSAIFI
4,25447-7854331,"(25447-7853701, 25447-7854331)",6.333996,16.773741,56.819996,217.589701,0.092777,0.092974
1,25447-7854100,"(25447-7853701, 25447-7854100)",7.137168,18.895060,10.250203,53.255432,0.189533,0.189935
3,25447-7853564C,"(25447-7853701, 25447-7853564C)",7.438358,19.690554,9.591571,42.940926,0.146912,0.147224
0,25447-7854587,"(25447-7853701, 25447-7854587)",7.721293,20.437838,8.910294,32.882838,0.083716,0.083894
6,25447-7854719CF,"(25447-7853701, 25447-7854719CF)",7.757801,20.534260,7.197805,25.419148,0.090247,0.090439
5,25447-7854108,"(25447-7853701, 25447-7854108)",7.766928,20.558367,7.893230,28.503394,0.076458,0.076620
7,25447-7854290CF,"(25447-7853701, 25447-7854290CF)",7.830817,20.727108,2.857942,11.801843,0.116626,0.116874
2,25447-7853335,"(25447-7853701, 25447-7853335)",7.858198,20.799425,3.086376,12.171874,0.084811,0.084991
8,25447-7854563CF,"(25447-7853701, 25447-7854563CF)",7.858198,20.799425,2.015360,8.083598,0.127704,0.127975


###7.2- Densely Urbanized System (PED SUbstation)

####7.2.1- Reading Graph from Database

In [19]:
#Read substation equivalent graph data from github repo
url = 'https://raw.githubusercontent.com/LSF-IEE/DistRel/main/PED.json'
with urllib.request.urlopen(url) as page:
    G = nx.node_link_graph(json.load(page))

#Number of consumers connected at each equiv. node (acquired from utility database)
n_cons = {x:G.nodes[x]['nuc_bt'] + G.nodes[x]['nuc_mt'] for x in G.nodes}

#Failure rate of each equiv. node [failures/yr] (acquired from utility database)
r_fail = {x:G.nodes[x]['tf'] for x in G.nodes}

#Permanent fault factor of each equiv. node [permanent faults/ total faults] (acquired from utility database)
f_perm = {x:G.graph['fator_fp'] for x in G.nodes}

#Emergency Response Times
#utL - average fault localization time [h]
#utR - average repair time [h]
#utT - average circuit section transfer time [h]
er_times = {x:{'utL': (G.graph['TMP']+G.graph['TMD']), 'utR': G.graph['TME'], 'utT': G.graph['TME']} for x in G.nodes}

#List of substation nodes (acquired from utility database)
n0_list = [x[0] for x in nx.get_node_attributes(G, "pac").items() if x[1] == True] 

#Definition of protection devices
devices = {'prot': ['22','29','32','33','34','35','36'], #Protection devices
                'rel': ['29','32','33','34','35','36'], #Prot. dev. w/ reclosing capacity
                'coord': ['32']} #Prot. dev. w/ coordinated reclosing capacity

#Standard deviation for emergency response times (acquired from utility database)
[std_tL,std_tR,std_tT] = [G.graph['TMPstd'],G.graph['TMDstd'],G.graph['TMDstd']]

#Minumum and maximum values for emergency response times (acquired from utility database)
[utL_min,utL_max] = [0.1*(G.graph['TMPmin'] + G.graph['TMDmin']), 1.3*(G.graph['TMPmax'] + G.graph['TMDmax'])]
[utR_min,utR_max] = [0.7*G.graph['TMEmin'], 1.3*G.graph['TMEmax']]
[utT_min,utT_max] = [0.7*G.graph['TMEmin'], 1.3*G.graph['TMEmax']]

#Historical Indexes Data (acquired from utility database)
SAIFImv_h = G.graph['FECp_h']
SAIDImv_h = G.graph['DECp_h']

Dref = G.graph['TMP'] + G.graph['TMD'] + G.graph['TME'] #mean service restoration time (acquired from utility database)
pbase = 1 #base power in kW for p.u. conversion (from load data acquired from utility database)
dt = 0.25 #time between samples in load curve, in hours (15 min => 0.25)
number_load_points = 96

####7.2.2- Calculate Reference Scenario

In [20]:
#No MFR at this momment             
nMFR = '' #node where the MFR is located
eIS = [] #edge where the islanding switch is located
mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode

#Create graph Gnom which is G excluded NO edges
Gnom = deepcopy(G)
Gnom.remove_edges_from([k for k,v in nx.get_edge_attributes(G, "P_N_OPE").items() if v == 'NO'])

#Remove isolated nodes (necessary fue to error in utility database)
G.remove_nodes_from([node for (node, val) in Gnom.degree() if val == 0])

#Calculate the shortest paths from each equiv. node to the substation nodes
dic_sp = [nx.single_source_shortest_path(Gnom, source) for source in n0_list]

#Create graph Gnosub which is G excluded substation nodes
Gnosub = deepcopy(G)
Gnosub.remove_nodes_from(n0_list)

#Initialize NFEM with all elements as permanently interrupted
Mf = {j:{i: {'state':'5', 'fi':0, 'di':0} for i in Gnosub.nodes} for j in Gnosub.nodes}

#Find the substation nodes that are connected to each equiv. node and their
#respective path
sub_vars = {n: find_sub_path(n) for n in G.nodes}

#Populate NFEM and the expected failure rate associated to each element
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    [state,fi] = NFE(G,nf,nh)
    Mf[nf][nh]['state'] = state
    Mf[nf][nh]['fi'] = fi

#Failure rate calibration from historical data
SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
print("SAIFImv_c before adjustment = {:.2f}".format(SAIFImv_c))

f_adj = SAIFImv_h/SAIFImv_c
print("Failure rate adjust factor = {:.2f}".format(f_adj))

for n in Gnosub.nodes:
  r_fail[n] = round(r_fail[n]*f_adj,2)
print("Adjusted failure rates = {}".format(list(r_fail.values())))

#Vulnerability and Contribution with respect to SAIFI
v = [round(sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nh in Gnosub.nodes]
p = [round(sum([Mf[nf][nh]['fi'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIFImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIFI = {}".format(v))
print("Relative Nodal Contribution w.r.t. SAIFI = {}".format(p))

#Calculate the expected failure rate again using calibrated failure rate param.
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    E = Mf[nf][nh]['state']
    edg_path_nf = sub_vars[nf][2]
    Mf[nf][nh]['fi'] = SAIDI_contr(G,E,nf,nh,edg_path_nf)

SAIFImv_c = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Calculate the initial expected outage time (iteration 0)
for nf in Gnosub.nodes:
  for nh in Gnosub.nodes:
    Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)

SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])

#Vulnerability and Contribution with respect to SAIDI
w = [round(sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nh in Gnosub.nodes]
d = [round(sum([Mf[nf][nh]['di'] for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])/SAIDImv_c,2) for nf in Gnosub.nodes]
print("Relative Nodal Vulnerability w.r.t. SAIDI = {}".format(w))
print("Relative Nodal Contribution w.r.t. SAIDI = {}".format(d))

#Calculate initial error in SAIDI calculation
eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)
print("\neSAIDI before adjust = {:.2f} %".format(eSAIDI*100))

#Emergency response time calibration from historical data
k_A = 0.1
eSAIDI_max = 0.01
it_mx = 100 #Maximum number of iterations

print("\nIterative process:")
iter = 0
while abs(eSAIDI) > eSAIDI_max:
  print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

  #Update emergency response times
  for x in Gnosub.nodes:
    er_times[x]['utL'] = var_check(er_times[x]['utL'] - eSAIDI*std_tL*k_A,utL_min,utL_max)
    er_times[x]['utR'] = var_check(er_times[x]['utR'] - eSAIDI*std_tR*k_A,utR_min,utR_max)
    er_times[x]['utT'] = var_check(er_times[x]['utT'] - eSAIDI*std_tT*k_A,utT_min,utT_max)

  #Update expected outage time
  for nf in Gnosub.nodes:
    for nh in Gnosub.nodes:
      Mf[nf][nh]['di'] = outage_dur(Mf,nf,nh)
  
  #Update SAIDImv_c and eSAIDI
  SAIDImv_c = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
  eSAIDI = round((SAIDImv_c - SAIDImv_h)/SAIDImv_h,4)

  #Next iteration
  if iter < it_mx:
    iter = iter + 1
  else:
    break

print("It. no. {}, utL = {}, utT = {}, utR = {}, SAIDImv_c = {}, eSAIDI = {}".format(iter,round(er_times[list(Gnosub.nodes)[0]]['utL'],4),round(er_times[list(Gnosub.nodes)[0]]['utT'],4),round(er_times[list(Gnosub.nodes)[0]]['utR'],4), round(SAIDImv_c,2), eSAIDI))

print("\nAdjusted SAIDImv_c = {:.2f}".format(SAIDImv_c))
print("Adjusted SAIFImv_c = {:.2f}".format(SAIFImv_c))

SAIFImv_c before adjustment = 2.86
Failure rate adjust factor = 1.50
Adjusted failure rates = [1.07, 3.91, 2.13, 2.04, 8.73, 2.4, 2.3, 2.23, 5.29, 1.5, 2.35, 2.59, 0.02, 0.96, 0.19, 0.62, 0.77, 0.92, 3.98, 6.23, 2.61, 0.02, 3.0, 3.33, 3.07, 2.74, 1.75, 0.1, 2.48, 0.51, 2.92, 4.49, 0.83, 1.2, 0.61, 1.06, 0.82, 0.3, 2.68, 0.63, 0.13, 4.88, 0.68, 1.32, 1.14, 0.23, 0.23, 2.17, 3.85, 0.92, 0.44, 0.06, 0.79, 2.11, 0.52, 0.08, 0.73, 0.07, 2.89, 2.29, 0.04, 0.89, 0.04, 1.27, 1.03, 0.83, 3.61, 1.85, 0.91, 0.5, 0.34, 2.11, 2.44, 3.51, 2.32, 0.04, 1.19, 3.22, 1.54, 1.56, 1.85, 0.12, 0.42, 0.71, 0.49, 3.45, 1.81, 4.83, 2.44, 2.35, 3.23, 0.05, 0.09, 0.59, 2.95, 0.41, 0.59, 0.02, 0.08, 0.06, 4.94, 0.15, 0.2, 0.68, 0.58, 4.24, 1.87, 0.61, 1.93, 2.36, 1.04, 0.02, 3.36, 1.48, 0.87, 0.65, 3.1, 0.47, 3.7, 0.42, 0.45, 0.06, 0.03, 1.17, 1.73, 0.02, 0.07, 0.06, 0.95, 0.37, 0.68, 1.59, 3.32, 0.09, 0.95, 0.62, 0.48, 1.46, 0.92, 0.55, 0.04, 2.2, 2.29, 1.07, 0.66, 0.77, 0.09, 1.61, 0.89, 0.78, 0.4, 0.59, 0.15, 

####7.2.3- Calculate nodal contribution, vulnerability and generate communities

In [21]:
#Calculate the set of most critical nodes (R)
Cblkp = {x:{'SAIDImv':{'ab':0, '%': 0},'SAIFImv':{'ab':0, '%': 0}} for x in Gnosub.nodes}

for nf in Gnosub.nodes:
    Cblkp[nf]['SAIFImv']['ab'] = sum([Mf[nf][nh]['fi'] for nh in Gnosub.nodes])/sum(n_cons.values())
    Cblkp[nf]['SAIFImv']['%'] = Cblkp[nf]['SAIFImv']['ab']/SAIFImv_c*100
    Cblkp[nf]['SAIDImv']['ab'] = sum([Mf[nf][nh]['di'] for nh in Gnosub.nodes])/sum(n_cons.values())
    Cblkp[nf]['SAIDImv']['%'] = Cblkp[nf]['SAIDImv']['ab']/SAIDImv_c*100

dfRES = pd.DataFrame.from_dict({(i,j): Cblkp[i][j] for i in Cblkp.keys() for j in Cblkp[i].keys()}, orient='index').sort_values(by=['%'],ascending = False)
dfSAIDImv = dfRES[dfRES.index.get_level_values(1) == 'SAIDImv'].droplevel(1).reset_index()
dfSAIDImv['cum'] = dfSAIDImv['%'].cumsum()
idx_50 = dfSAIDImv[dfSAIDImv.cum >= 50].index[0]
n_crit_list = list(dfSAIDImv.loc[0:idx_50]['index'])
print("Set of most critical nodes (R): {}".format(n_crit_list))

#Calculate the set of most vulnerable nodes (U)
Vblkp = {x:{'SAIDImv':{'ab':0, '%': 0},'SAIFImv':{'ab':0, '%': 0}} for x in Gnosub.nodes}
for nh in Gnosub.nodes:
    Vblkp[nh]['SAIFImv']['ab'] = sum([Mf[nf][nh]['fi'] for nf in Gnosub.nodes])/sum(n_cons.values())
    Vblkp[nh]['SAIFImv']['%'] = Vblkp[nh]['SAIFImv']['ab']/SAIFImv_c*100
    Vblkp[nh]['SAIDImv']['ab'] = sum([Mf[nf][nh]['di'] for nf in Gnosub.nodes])/sum(n_cons.values())
    Vblkp[nh]['SAIDImv']['%'] = Vblkp[nh]['SAIDImv']['ab']/SAIDImv_c*100

dfRES_vun = pd.DataFrame.from_dict({(i,j): Vblkp[i][j] for i in Vblkp.keys() for j in Vblkp[i].keys()}, orient='index').sort_values(by=['%'],ascending = False)
dfSAIDImv_vun = dfRES_vun[dfRES_vun.index.get_level_values(1) == 'SAIDImv'].droplevel(1).reset_index()
dfSAIDImv_vun['cum'] = dfSAIDImv_vun['%'].cumsum()
idx_50_vun = dfSAIDImv_vun[dfSAIDImv_vun.cum >= 50].index[0]
n_vun_list = list(dfSAIDImv_vun.loc[0:idx_50_vun]['index'])
print("Set of most vulnerable nodes (U): {}".format(n_vun_list))

#Divide the graph into communities using the CNM algorithm
communities = greedy_modularity_communities(G)

Set of most critical nodes (R): ['12056-7017891', '12068-7042344', '13708-11325770', '13708-9570917', '12066-7041703', '12068-7019109', '12065-11522090', '12066-7041499', '13708-9571648', '12068-7042652', '12066-11857919', '13708-9571869', '12066-7041232C', '12122-7219325', '12066-7041013', '12066-7041638', '12061-7027442', '12054-7008821', '12122-7218775', '12061-7027508', '12066-7041634', '12068-7042359', '12064-7030097', '12054-7008604', '12059-7022502', '12056-7018171', '12066-12195175', '12065-7040546']
Set of most vulnerable nodes (U): ['13708-9571683CF', '12066-7041703', '13708-9571731CF', '13708-9571536CF', '13708-9571735CF', '13708-9769292C', '13708-9571041CF', '13708-9571737CF', '13708-9571353CF', '13708-9571043CF', '12066-7041850CF', '13708-9571033CF', '12068-7019149CF', '13708-9571829CF', '12066-7041986CF', '12066-7041994CF', '13708-9571913', '12068-7018983CF', '12066-7041611CF', '12051-7007844CF', '12066-7041499', '12066-7042174CF', '12058-7019531', '12061-7027361', '12066

####7.2.4- Graph Visualization

In [22]:
communities_dict = {}
shape_dict = {}

for i in range(len(communities)):
  for node in list(communities[i]):
    communities_dict[node] = i

for node in G.nodes:
  if G.nodes[node]['pac']:
    shape_dict[node] = 'star'
  else:
    if node in n_crit_list:
      shape_dict[node] = 'triangle'
    else:
      if node in n_vun_list:
        shape_dict[node] = 'square'
      else:
        shape_dict[node] = 'dot'

nx.set_node_attributes(G, communities_dict, name='group')
nx.set_node_attributes(G, shape_dict, name='shape')

nt = Network(notebook=True, cdn_resources='in_line', bgcolor='white',font_color="red")
nt.from_nx(G)
nt.show('{}-ref.html'.format('PED'))
display(HTML('{}-ref.html'.format('SSB')))

PED-ref.html


####7.2.5- Generate Scenarios for MFR allocation

In [23]:
nMFR_list = []
eIS_list = []

#For each critical node
for n_crit in n_crit_list:
  #Nodes in the n_crit community
  nodes_com_n_crit = set([x for x in communities if n_crit in x][0]) 
   
  for node in nodes_com_n_crit:
    #For each node in n_crit community, list its neighbors
    neigh_list = list(G.neighbors(node))
    for neigh in neigh_list:
      #If the neighbor is from another community, and is not a substation node, then the node is a possible nMFR, and node-neigh a possible eIS
      if not(neigh in nodes_com_n_crit):
        if not(neigh in sub_vars[n_crit][1]):
          nMFR_list.append(neigh)
          eIS_list.append((node,neigh))

  #Check if there is a vulnerable node in the critical node community
  for node in nodes_com_n_crit:
    if node in n_vun_list:
      #If existis, check if the supply path passes through the critical node
      if n_crit in sub_vars[node][1]:
        #If passes, determine the edge that leaves the critical node towards the vulnerable node
        nMFR_list.append(sub_vars[node][1][sub_vars[node][1].index(n_crit)-1])
        eIS_list.append((n_crit,sub_vars[node][1][sub_vars[node][1].index(n_crit)-1]))
      else:
        #If it doesn't passes
        for nd in sub_vars[node][1][1:]:
          neigh_list = list(G.neighbors(nd))
          for neigh in neigh_list:
            if not(neigh in nodes_com_n_crit):
              nMFR_list.append(neigh)
              eIS_list.append((nd,neigh))

#Remove duplicates
scen_df = pd.DataFrame.from_dict(dict(zip(nMFR_list, eIS_list)), orient='index').reset_index().drop_duplicates()
nMFR_list = list(scen_df['index'])
eIS_list = list((list(scen_df[0])[k],list(scen_df[1])[k]) for k in range(len(scen_df)))

####7.2.6- Run Reliability Assessment for each possible scenario

In [24]:
%%time

SAIFImv_n_list = []
SAIDImv_n_list = []
P_mfr_list = []
E_mfr_list = []
rSAIDI_list = []
rSAIFI_list = []
for edge,nMFR in zip(eIS_list,nMFR_list):  
  #First, check if the MFR/IS is a valid combination
  eIS  = [edge]
  if edge in sub_vars[nMFR][2] or edge[::-1]  in sub_vars[nMFR][2]:
    
    mg_isld_time = 0.0 #transition time from grid connected mode to islanded mode
    Mf_mod = deepcopy(Mf)

    #Populate the new NFEM and update the expected failure rate associated to the nodes islanded in microgrid
    for nf in Gnosub.nodes:
      for nh in Gnosub.nodes:          
        if (nf != nh):
          #If nh is permanently interrupted in the reference case
          if (Mf_mod[nf][nh]['state'] == '5'): 
            #If it is possible to transfer nh to operate in islanded MG
            if check_transfer_mg(G,nf,nh):
              Mf_mod[nf][nh]['state'] = '4'
              edg_path_nf = sub_vars[nf][2]
              Mf_mod[nf][nh]['fi'] = SAIDI_contr(G,'4',nf,nh,edg_path_nf)

    #Update expected outage time
    for nf in Gnosub.nodes:
      for nh in Gnosub.nodes:
        Mf_mod[nf][nh]['di'] = outage_dur(Mf_mod,nf,nh)

    SAIFImv_n = sum([Mf_mod[nf][nh]['fi'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
    SAIDImv_n = sum([Mf_mod[nf][nh]['di'] for nf in Gnosub.nodes for nh in Gnosub.nodes])/sum([n_cons[n] for n in Gnosub.nodes])
    SAIFImv_n_list.append(SAIFImv_n)
    SAIDImv_n_list.append(SAIDImv_n)
    print("\nnMFR: {}, eIS: {}\nSAIFImv_n: {:.2f}, SAIDImv_n = {:.2f} h".format(nMFR,eIS,SAIFImv_n,SAIDImv_n))

    #Calculate Power and Energy Requirements
    #First, determine which equivalent nodes are part of the microgrid
    Gisld = deepcopy(Gnom)
    Gisld.remove_edge(*eIS[0]) 
    mg_nodes = list(nx.single_source_shortest_path(Gisld, nMFR).keys())
    print("Nodes in MG: {}".format(mg_nodes))

    #Then, calculate the equivalent load curve
    eq_ld_curve = np.zeros(number_load_points)
    for node in mg_nodes:
      eq_ld_curve = eq_ld_curve + np.array(G.nodes[node]['eq_load'])
    
    eq_ld_curve = pd.DataFrame({'equiv':eq_ld_curve})

    #Determine the Moving Average and Calculate Pmfr and Emfr
    [P_mfr,E_mfr] = size_MFR(eq_ld_curve,Dref,dt,pbase)
    P_mfr_list.append(P_mfr)
    E_mfr_list.append(E_mfr)
    print("Pmfr = {:.2f} kW and Emfr = {:.2f} kWh".format(P_mfr,E_mfr))

    #Calculate rSAIDI and rSAIFI
    if E_mfr !=0 :
      rSAIDI = (SAIDImv_c - SAIDImv_n)/SAIDImv_c*100/E_mfr
      rSAIFI = (SAIFImv_c - SAIFImv_n)/SAIFImv_c*100/E_mfr
    else:
      rSAIDI = 0
      rSAIFI = 0
    rSAIDI_list.append(rSAIDI)
    rSAIFI_list.append(rSAIFI)
    print("rSAIDI = {:.2f} %/kWh and rSAIFI = {:.2f} %/kWh".format(rSAIDI,rSAIFI))

  else:
    SAIFImv_n_list.append('')
    SAIDImv_n_list.append('')
    P_mfr_list.append('')
    E_mfr_list.append('')
    rSAIDI_list.append('')
    rSAIFI_list.append('')
    print("IS not in MFR's path!")

dfMG = pd.DataFrame({'nMFR': nMFR_list,'eIS': eIS_list,'SAIFImv_n': SAIFImv_n_list,'SAIDImv_n': SAIDImv_n_list, 'P_mfr': P_mfr_list, 'E_mfr': E_mfr_list, 'rSAIDI': rSAIDI_list, 'rSAIFI': rSAIFI_list})
dfMG['SAIDImv_n'] = pd.to_numeric(dfMG.SAIDImv_n, downcast='float')
dfMG['SAIFImv_n'] = pd.to_numeric(dfMG.SAIFImv_n, downcast='float')
dfMG = dfMG.dropna()

print('\nExecution Time:')


nMFR: 12056-7018450, eIS: [('12056-7018477C', '12056-7018450')]
SAIFImv_n: 4.21, SAIDImv_n = 4.96 h
Nodes in MG: ['12056-7018450', '12056-7018273', '12056-7018291', '12056-7018453', '12056-7017899CF', '12056-7018039CF', '12056-7017950', '12056-7018026', '12056-7017495', '12056-7018199', '12056-7018145CF', '12056-7018143CF', '12056-7018215CF', '12056-7017614CF', '12056-7018169CF', '12056-7018181CF', '12056-7018138', '12056-7017787', '12056-12275439', '12056-7017522CF', '12056-7018486CF', '12056-7018484CF', '12056-7018492CF', '12056-7017520CF', '12056-7017528CF', '12056-11679122CF', '12056-7017789CF', '12056-7017497CF', '12056-7018301', '12056-7017483CF', '12056-7018496CF', '12056-7018500CF']
Pmfr = 769.54 kW and Emfr = 5649.85 kWh
rSAIDI = 0.00 %/kWh and rSAIFI = 0.00 %/kWh

nMFR: 12056-7018149, eIS: [('12056-7017891', '12056-7018149')]
SAIFImv_n: 4.25, SAIDImv_n = 5.02 h
Nodes in MG: ['12056-7018149', '12056-7017972', '12056-7018171', '12056-7018498CF', '12056-7017893CF', '12056-70183

In [25]:
dfMG.sort_values(by=['SAIDImv_n'])

,nMFR,eIS,SAIFImv_n,SAIDImv_n,P_mfr,E_mfr,rSAIDI,rSAIFI
8,13708-9572059,"(13708-11325770, 13708-9572059)",4.122003,4.853972,730.586902,5353.544121,0.000825,0.000771
76,12066-7041634,"(12066-7041404, 12066-7041634)",4.133750,4.867764,2539.066644,18141.405692,0.000229,0.000213
93,12066-7041404,"(12066-7041638, 12066-7041404)",4.139568,4.875179,2556.782666,18288.960305,0.000219,0.000203
2,12056-7018477F,"(12056-7017891, 12056-7018477F)",4.172889,4.918036,840.301702,6166.826844,0.000512,0.000478
60,12066-11857919,"(12066-7041634, 12066-11857919)",4.175135,4.920256,1655.039741,11273.761329,0.000276,0.000257
...,...,...,...,...,...,...,...,...
69,12066-7041521CF,"(12066-11857919, 12066-7041521CF)",4.299540,5.078375,0.0,0.0,0,0
63,12066-11857936CF,"(12066-11857919, 12066-11857936CF)",4.299540,5.078375,0.0,0.0,0,0
62,12066-7041643CF,"(12066-11857919, 12066-7041643CF)",4.299540,5.078375,0.0,0.0,0,0
96,12066-7041404CF,"(12066-7041404, 12066-7041404CF)",4.299540,5.078375,0.0,0.0,0,0


###7.3- Timing Considerations

In [26]:
#Considering the PED system:
n_scen_red = 154
total_exec_time_red = 40 #approximately 40 mins

time_scen = total_exec_time_red/n_scen_red #Execution time per scenario evaluation
print('Execution time per scenario: {:.2f} minutes'.format(time_scen))

#Now, considering an exhaustive search approach:
n_scen_exh = len(Gnosub.nodes)
total_exec_time_exh = n_scen_exh*time_scen
print('Expected total execution time in exhaustive search: {:.2f} h'.format(total_exec_time_exh/60))

Execution time per scenario: 0.26 minutes
Expected total execution time in exhaustive search: 5.69 h
